In [1]:
import numpy as np
from scipy.special import factorial
import pandas as pd
import time

## Task 1 

In [2]:
def generate_primes(n,batch_size = 10):
    
    arr = np.array( [ 2,3 ] )

    step = 5 if n <= 50 else int(round( n/batch_size , -1)/2)
    c = 1
    while len(arr) < n:
        rng_arr = np.reshape(np.arange( c, c+step ), (-1, step ) )
        primes_arr = np.concatenate([(6*rng_arr - 1) ,(6*rng_arr + 1) ]).flatten('F')
        # Wilson's theorem - (n-1)! + 1 is divisible to n
        #res = primes_arr[ ((factorial(primes_arr-1, exact =  True) + 1) % primes_arr) == 0 ]
        mask = np.apply_along_axis( lambda x,y : 0 not in ( x % y[y <= np.sqrt(x)] )   , axis=0, arr = primes_arr.reshape( -1 ,primes_arr.shape[0] ) ,y=np.concatenate([ arr, primes_arr ]) )       
        res = primes_arr[mask] 
        arr = np.concatenate( [ arr, res ] )
        c+= step
        
    return arr[:n]
    

In [3]:
'''
2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131,
137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199…
'''

'\n2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,\n61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131,\n137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199…\n'

In [7]:
tac = time.time()
t1 = generate_primes( 10001 )
print(t1.shape)
print( t1 )
print( time.time() - tac )

(10001,)
[     2      3      5 ... 104723 104729 104743]
0.6760604381561279


In [ ]:
# Provide list of every number in the sequence till 10001st in csv file
pd.DataFrame( t , columns = ["Primes"]).to_csv( "Primes.csv", index = False )

In [ ]:
#  Sum 1st, 2nd, 101st, 1001st, and 10001st numbers from aforementioned sequence
np.sum( t[ [ 0 ,1,100,1000,10000  ] ] )

## Task 2

In [8]:
n_sh = 100
n = np.arange(  1, n_sh +1   ).reshape(  n_sh , -1  )
b = np.arange(  1, 1001  ).reshape( -1, 1000  )
c = np.arange(  1, 1001  ).reshape( -1, 1000 )
y = generate_primes( 11000 )

In [9]:
def cons( x ):
    a = np.nonzero(x)[0]
    con = np.diff(np.concatenate(([-1],) + np.nonzero(np.diff( a  ) != 1) + ([len(  a  )-1],)))
    return con.max()

tic = time.time()
v1 = ( (np.square(n) + (n * b))[..., np.newaxis]  + c ).reshape( n_sh , 1000000 )
mask = np.isin( v1,y )
consenq = np.apply_along_axis( cons ,arr = mask , axis = 0   )
print( time.time() - tic ) 

28.70930290222168


In [10]:
np.argsort( consenq )

array([766709, 812849, 812847, ...,   4046,   2042,     40], dtype=int64)

In [17]:
mult_BC = []

for i in np.argsort( consenq )[ -3: ]:
    arg = i + 1 
    B = ( arg // 1000 )+1
    C = ( arg % 1000 )
    mult_BC.append( B * C )
    print( "B =",B,";","C =",C )

# show result
print( "That 3 nums:" , mult_BC, " Sum :", sum(mult_BC) )

B = 5 ; C = 47
B = 3 ; C = 43
B = 1 ; C = 41
That 3 nums: [235, 129, 41]  Sum : 405


In [18]:
#testing
t = np.square(n) + n*1 + 41
mask = np.isin( t,y )
a = np.nonzero(mask)[0]
con = np.diff(np.concatenate(([-1],) + np.nonzero(np.diff(a) != 1) + ([len(a)-1],)))
con.max()


39

## Task 3 

In [ ]:

def snake_array(arr_sh):
    
    m = np.arange( arr_sh**2 ,0,-1)
    arr = np.zeros( ( arr_sh , arr_sh ) )
    
    i1 ,sh =  0, arr_sh
    for i in range(  (arr_sh-1)//2  ):
        if i > 0:
            arr[i,i:-i] = m[ i1 : i1+sh ][::-1]
            arr[ i:-i ,i ] = m[ (i1-1+sh) : (i1-1+(2*sh)) ]
            arr[-(i+1), i:-i ] =  m[ (i1-2+(2*sh)) : (i1-2+(3*sh))  ]
            arr[i+1:-i,-(i+1)] = m[ (i1-3+(3*sh)) : (i1-4+(4*sh))   ][::-1]
        else:
            arr[0,:] = m[ i1 : i1+sh ][::-1]
            arr[ :, 0 ] = m[ (i1-1+sh) : (i1-1+(2*sh)) ]
            arr[-1,:] =  m[ (i1-2+(2*sh)) : (i1-2+(3*sh))  ]
            arr[1:,-1] = m[ (i1-3+(3*sh)) : (i1-4+(4*sh))   ][::-1]

        i1 = (i1-4+(4*sh))
        sh -= 2

    arr[ (arr_sh-1)//2 , (arr_sh-1)//2 ] = 1
    return arr



In [ ]:
tac = time.time()
for num in [7,13,19]:
    print( "-"*20 , num , "-"*20 )
    arr = snake_array( num )
    diag1 = np.diag(arr)
    diag2 = np.diag(np.fliplr(arr))
    print( "Sum of diagonals:", np.sum( [ diag1, diag2 ] ) - 1  )
    print( "Multiplication numbers on the corners:", diag1[0]*diag1[-1]*diag2[0]*diag2[-1] )
    
print( time.time() - tac )